<a href="https://colab.research.google.com/github/adtitovich/dsml-17/blob/main/m2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание задания:
В домашнем задании необходимо применить полученные знания в теории оптимизации и машинном обучении для реализации логистической регрессии.

#Этапы работы:
**1. Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.**

In [86]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix

# загружаем датасет с ириcами
data = load_iris()
# переводим в датафрейм
df = pd.DataFrame(data = data['data'], columns = data['feature_names'])
# добавляем целевую переменную
df['target'] = data['target']

df.head(10)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
5,5.4,3.9,1.7,0.4,0
6,4.6,3.4,1.4,0.3,0
7,5.0,3.4,1.5,0.2,0
8,4.4,2.9,1.4,0.2,0
9,4.9,3.1,1.5,0.1,0


In [87]:
df.shape, np.unique(df.target)

((150, 5), array([0, 1, 2]))

In [88]:
# Убирвем данные по setosa, оставляем versicolor=0; virginica=1

df['target'] -= 1 
df = df[df.target != -1]
# переиндексуем
df.reset_index(drop=True, inplace=True)
df.shape, np.unique(df.target)

((100, 5), array([0, 1]))

**2. Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции.**

In [89]:
# объявим переменные
X = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
X.insert(0,'x0', np.ones(X.shape[0])) # добавим столбец с единицами

y = df['target']

# вектор начальных весов (начальное значение - 0)
thetas = np.zeros(X.shape[1])

# кол-во наблюдений
n = X.shape[0]

#  список для записи размера ошибки функции потерь
loss_history = []

# коэффициент скорости обучения
learning_rate = 0.01

# Кол-во итераций
iter = 5000

In [90]:
# сигмоида
def stable_sigmoid(z):
  if z >= 0:
      return 1 / (1 + np.exp(-z))
  else:
      return np.exp(z) / (np.exp(z) + 1)

In [91]:
# логистическая регрессия
def h(x, thetas):
  z = np.dot(x, thetas)
 
  return np.array([stable_sigmoid(value) for value in z])

In [92]:
# функция бинарной кросс-энтропии
def logloss(y, y_pred):
  # добавляем 1e-10 чтобы избежать ошибки при log(0)
  return -np.mean(y * np.log(y_pred + 1e-10) + (1 - y) * np.log(1 - y_pred + 1e-10))


In [93]:
# градиент
def gradient(x, y, y_pred, n):
  return np.dot(x.T, (y_pred - y)) / n

**3. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.**

In [94]:
for _ in range(iter):
  # прогнозируемое значение с текущими весами
  y_pred = h(X, thetas)
  # уровень ошибки при текущем прогнозе
  loss_history.append(logloss(y, y_pred))
  # градиент
  grad = gradient(X, y, y_pred, n)
  # улучшаем веса модели
  thetas = thetas - learning_rate* grad

In [95]:
#веса модели и финальный уровень ошибки
thetas, loss_history[-1]

(array([-1.01082282, -1.82769772, -1.47718748,  2.58792586,  2.44175357]),
 0.22140727226810383)

**Оценим результат с помощью матрицы ошибок**

In [96]:
# предсказание к какому классу относится то или иное наблюдение
# если вероятность больше или равна 0,5 - отнесем наблюдение к классу 1, 
# в противном случае к классу 0
def predict(x, thetas):
  z = np.dot(x, thetas)
  probs = np.array([stable_sigmoid(value) for value in z])
  return np.where(probs >= 0.5, 1, 0)

In [97]:
# запишем прогноз класса
y_pred = predict(X, thetas)

In [98]:
#Матрица ошибок
pd.DataFrame(confusion_matrix(y, y_pred),
             columns = ['versicolor', 'virginica'],
             index = ['versicolor', 'virginica'])

,versicolor,virginica
versicolor,47,3
virginica,0,50
